In [4]:
import os

from bids import BIDSLayout

from glob import glob

from nipype.interfaces.io import BIDSDataGrabber
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import Function

In [7]:
bids_dir = os.path.join('/fetaldata')
mask_dir = os.path.join('/fetaldata','derivatives','manual_masks')

subject = '01'

layout = BIDSLayout(bids_dir,derivatives=mask_dir)
print(layout)


BIDS Layout: .../fetaldata | Subjects: 9 | Sessions: 10 | Runs: 57


In [9]:
from traits.api import *

import nibabel as nib

from nipype.utils.filemanip import split_filename
from nipype.interfaces.base import traits, isdefined, CommandLine, CommandLineInputSpec,\
    TraitedSpec, File, InputMultiPath, OutputMultiPath, BaseInterface, BaseInterfaceInputSpec

from nipype.interfaces.mixins import reporting



In [35]:
import nibabel as nib
import numpy as np
import skimage.measure


In [61]:
class StacksOrderingInputSpec(BaseInterfaceInputSpec):
    """Class used to represent inputs of the StacksOrdering interface.

    Attributes
    -----------
    input_masks <list<string>>
        Input brain masks on which motion is computed.

    See also
    --------------
    pymialsrtk.interfaces.preprocess.StacksOrdering

    """
    
    input_masks = InputMultiPath(File(desc='Input masks',mandatory=True))
    
    
class StacksOrderingOutputSpec(TraitedSpec):
    """Class used to represent outputs of the StacksOrdering interface.

    Attributes
    -----------
    stacks_order <string>
        Order of images' run-id to be used for reconstruction

    See also
    --------------
    pymialsrtk.interfaces.preprocess.StacksOrdering

    """
    
    stacks_order = traits.List(desc = 'Order of stacks')

    
class StacksOrdering(BaseInterface):
    """Runs the automatic ordering of stacks.

    This module is based on a centroid tracking of the the central of the brain mask.

    Examples
    --------
    >>> from pymialsrtk.interfaces.preprocess import StacksOrdering
    >>> stacksOrdering = StacksOrdering()
    >>> stacksOrdering.inputs.input_images = ['my_image_run-1.nii.gz', 'my_image_run-3.nii.gz', 'my_image_run-2.nii.gz']
    >>> stacksOrdering.run() # doctest: +SKIP

    """
    
    input_spec = StacksOrderingInputSpec
    output_spec = StacksOrderingOutputSpec
    
    m_stack_order = []
    
    def _run_interface(self, runtime): 
        try:
            self.m_stack_order = self._compute_stack_order(self.inputs.input_masks)
        except Exception:
            print('Failed')
            
            print(traceback.format_exc())
        return runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['stacks_order'] = self.m_stack_order
        return outputs
    
    def _compute_motion_index(self, in_file):
        """Function to compute the motion index.

        The motion index is computed from the inter-slice displacement of the centroid of the brain mask.  
        """
        central_third = True

        img = nib.load(in_file)

        voxelspacing = img.header['pixdim'][2]
        data = img.get_fdata()

        z = np.where(data)[2]        
        data=data[...,int(min(z)):int(max(z)+1)]

        if central_third:
            num_z=data.shape[2]
            center_z=int(num_z/2.)
            
            data=data[...,int(center_z-num_z/6.):int(center_z+num_z/6.)]

        centroid_coord=np.zeros((data.shape[2],2))

        for i in range(data.shape[2]):
            moments=skimage.measure.moments(data[...,i])
            centroid_coord[i,:]=[moments[0,1]/moments[0,0],moments[1,0]/moments[0,0]]

        centroid_coord = centroid_coord[~np.isnan(centroid_coord)]
        centroid_coord = np.reshape(centroid_coord,(int(centroid_coord.shape[0]/2),2))

        nSlices = data.shape[2]
        score = (np.var(centroid_coord[:,0])+np.var(centroid_coord[:,1])) / nSlices

        return score


    def _compute_stack_order(self, in_files):
        """Function to compute the stacks order.

        The motion index is computed for each mask. Stacks are ordered according to their motion index. 
        """
        motion_ind = []

        for f in in_files:
            motion_ind.append(self._compute_motion_index(f))
        
        motion_ind_sorted, images_ordered = (list(t) for t in zip(*sorted(zip(motion_ind, in_files))))
        run_order = [ int(f.split('run-')[1].split('_')[0]) for f in images_ordered]
        
        return run_order
        

In [62]:
stacksOrdering = StacksOrdering()

stacksOrdering.inputs.input_masks = ['/fetaldata/derivatives/manual_masks/sub-01/anat/sub-01_run-1_T2w_desc-brain_mask.nii.gz',
                        '/fetaldata/derivatives/manual_masks/sub-01/anat/sub-01_run-2_T2w_desc-brain_mask.nii.gz',
                         '/fetaldata/derivatives/manual_masks/sub-01/anat/sub-01_run-3_T2w_desc-brain_mask.nii.gz',
                        '/fetaldata/derivatives/manual_masks/sub-01/anat/sub-01_run-4_T2w_desc-brain_mask.nii.gz',
                         '/fetaldata/derivatives/manual_masks/sub-01/anat/sub-01_run-5_T2w_desc-brain_mask.nii.gz',
                        '/fetaldata/derivatives/manual_masks/sub-01/anat/sub-01_run-6_T2w_desc-brain_mask.nii.gz']

aa = stacksOrdering.run() # doctest: +SKIP
aa.outputs.stacks_order

[2, 5, 3, 4, 6, 1]

In [63]:
#Check if mandatory derivatives dataset_description.json exists in derivatives/mialsrtk.
# If not, it is created before running the workflow, otherwise BIDSDataGrabber is not happy and raises an error. 

mialsrtk_dataset_description_json = os.path.join(output_dir,'dataset_description.json')

print('Check for {}'.format(mialsrtk_dataset_description_json))
if not os.access(mialsrtk_dataset_description_json, os.R_OK):
    import json
    data = {'PipelineDescription':{'Name': 'MIAL Super-Resolution ToolKit', 
                                'Version': 'v2.0.0-beta', 
                                'CodeURL': 'https://github.com/sebastientourbier/mialsuperresolutiontoolkit'
                                  },
            'Name': 'MIAL Super-Resolution ToolKit',
            'BIDSVersion': '1.2.0'
           }
    os.makedirs(output_dir)
    with open(mialsrtk_dataset_description_json, 'w+') as outfile:
        json.dump(data, outfile, indent=4)
    print('File {} was created'.format(mialsrtk_dataset_description_json))
else:
    print('File {} already exists'.format(mialsrtk_dataset_description_json))
    

wf = Workflow(name="mon_stacksOrdering",base_dir=os.path.join(output_dir,'sub-{}'.format(subject),'nipype'))

bg = Node(interface=BIDSDataGrabber(infields = ['subject']),name='bids_grabber')
bg.inputs.base_dir = bids_dir
bg.inputs.subject = subject
bg.inputs.index_derivatives = True
bg.inputs.raise_on_empty = False
bg.inputs.output_query = {'T2ws': dict(suffix='T2w',datatype='anat',extensions=[".nii",".nii.gz"]),
                          'masks': dict(suffix='mask',datatype='anat',extensions=[".nii",".nii.gz"])}

stacksOrdering = Node(interface = StacksOrdering(),name='mon_stacksOrdering_module')

wf.connect(bg, "masks", stacksOrdering, "input_masks")

res = wf.run()

Check for /fetaldata/derivatives/mialsrtk/dataset_description.json
File /fetaldata/derivatives/mialsrtk/dataset_description.json already exists
201110-18:16:46,341 nipype.workflow INFO:
	 Workflow mon_stacksOrdering settings: ['check', 'execution', 'logging', 'monitoring']
201110-18:16:46,346 nipype.workflow INFO:
	 Running serially.
201110-18:16:46,347 nipype.workflow INFO:
	 [Node] Setting-up "mon_stacksOrdering.bids_grabber" in "/fetaldata/derivatives/mialsrtk/sub-01/nipype/mon_stacksOrdering/bids_grabber".
201110-18:16:46,350 nipype.workflow INFO:
	 [Node] Running "bids_grabber" ("nipype.interfaces.io.BIDSDataGrabber")
201110-18:16:46,858 nipype.workflow INFO:
	 [Node] Finished "mon_stacksOrdering.bids_grabber".
201110-18:16:46,859 nipype.workflow INFO:
	 [Node] Setting-up "mon_stacksOrdering.mon_stacksOrdering_module" in "/fetaldata/derivatives/mialsrtk/sub-01/nipype/mon_stacksOrdering/mon_stacksOrdering_module".
201110-18:16:46,861 nipype.workflow INFO:
	 [Node] Cached "mon_stack